<a href="https://colab.research.google.com/github/rar8393/NLG_tests/blob/main/NLG_opt_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 8.0 MB/s 
     |████████████████████████████████| 163 kB 71.0 MB/s 
     |████████████████████████████████| 6.6 MB 35.3 MB/s 


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

#model = AutoModelForCausalLM.from_pretrained("facebook/opt-30b", torch_dtype=torch.float16).cuda()

# the fast tokenizer currently does not work correctly
#tokenizer = AutoTokenizer.from_pretrained("facebook/opt-30b", use_fast=False)

In [7]:
from transformers import pipeline, set_seed

set_seed(32)
generator = pipeline('text-generation', max_new_tokens=250, model="facebook/opt-350m", do_sample=True)
generator("Hello, I'm am conscious and")

[{'generated_text': "Hello, I'm am conscious and curious who you are, and if you have anything to say for now. I'll be trying to make some money this year, and am interested in learning more about finance and investing. I'm interested in the idea of building it into an investment, and maybe doing my first home equity loan.  So, any pointers on how to go about getting started? Maybe I can contact you?"}]

In [9]:
event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '
generator(event_txt + sick_txt)

[{'generated_text': 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will ive a great community gathering!\n\nAmen! I am in attendance at the webinars, it is a great thing!\nAmen. There is too much at stake for just an "official" event, so I am very glad you have organized it.\n\nThe flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will ive a great community gathering!\n\nI also attended the webinar. It was a great time. We will now take suggestions for next year and be working on this soon. We started an article that describes how we are devel

In [10]:
event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers.  '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '
generator(event_txt + sick_txt)

[{'generated_text': 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers.  The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will  be the first time we will be holding a webinar.  For further information can PM or email.\nWould love to help, please send us a PM'}]

In [43]:
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m").to(device)#, torch_dtype=torch.float16).cuda()

# the fast tokenizer currently does not work correctly
#tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m", use_fast=False)

In [41]:
from transformers import GPT2Tokenizer, OPTForCausalLM

tokenizer = GPT2Tokenizer.from_pretrained("facebook/opt-350m")
model = OPTForCausalLM.from_pretrained("facebook/opt-350m")#.to(device)

In [42]:
inputs = tokenizer("Hello, my dog is cute and ", return_tensors="pt")
generation_output = model.generate(**inputs, max_length=64)
tokenizer.batch_decode(generation_output, skip_special_tokens=True)

["Hello, my dog is cute and  I love him. I'm a little worried about him. I'm not sure if he's a good fit for me. I'm not sure if he's a good fit for me. I'm not sure if he's a good fit for me.\nI'm not"]

In [36]:
input_ids = tokenizer("Hello, my dog is cute and ", return_tensors="pt").input_ids #, return_tensors="pt"
outputs = model.generate(input_ids, do_sample=False, max_length=30)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

["Hello, my dog is cute and  I'm a little shy but I love her. She's a little girl and she's a little girl"]

In [53]:
gen_text = "Hello, my dog is cute and "
text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

In [54]:
text

"Hello, my dog is cute and  I'm a little shy but I love her. She's a little girl and she's a little girl"

In [55]:
text.replace(gen_text, "" )

" I'm a little shy but I love her. She's a little girl and she's a little girl"

In [74]:
import importlib
import generated_text_metrics
importlib.reload(generated_text_metrics)

<module 'generated_text_metrics' from '/content/generated_text_metrics.py'>

In [75]:
from generated_text_metrics import generated_text_metric, generation_sentence_simialrity

In [77]:
text

"Hello, my dog is cute and  I'm a little shy but I love her. She's a little girl and she's a little girl"

In [78]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [81]:
doc_sent = nlp(text)
for sent in doc_sent.sents:
  print('new')
  print(sent)

new
Hello, my dog is cute and  I'm a little shy
new
but I love her.
new
She's a little girl and she's a little girl


In [76]:
generated_text_metric(text)

{'number_ents': 0,
 'ents': [],
 'number_nouns': 3,
 'nouns': ['dog', 'girl', 'girl'],
 'number_sents': 3,
 'avg_sent_len': 9.666666666666666,
 'sent_repeat_score': [0.07692307692307687, 0.0, 0.4545454545454546]}

In [63]:
generated_txt = text.replace(gen_text, "" )

In [65]:
generation_sentence_simialrity(gen_text, generated_txt)

/content/generated_text_metrics.py:44: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity.append(gen_doc.similarity(sent))
/content/generated_text_metrics.py:46: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one 

[0.13218830379839042, 0.22562848031520844, 0.3252957761287689]